In [1]:
import json
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss


In [2]:
def read_file(path):
    with open(f"{path}") as f:
        objects = json.load(f)
    return objects

In [3]:
jsons_dict = read_file("../../tuning_inference_adapted/data.json")

In [4]:
df = pd.DataFrame(jsons_dict)
df.to_csv('database_context.csv', index=False)
df.head()

,id,category,question,context,answer,doc_len
0,0,Regulamento,Quais são as causas para perder o vínculo com ...,,São elas: não efetuar matrícula em disciplinas...,17
1,1,Regulamento,Quando o desvínculo acontece?,,O desvínculo acontece após o último dia de imp...,4
2,2,Regulamento,O que é o procedimento de desvinculação?,,O procedimento de desvinculação é o processo d...,7
3,3,Regulamento,Quanto tempo o discente tem para requerer o re...,,O discente tem 10 dias úteis a contar da ciênc...,9
4,4,Regulamento,Onde o recurso deve ser encaminhado?,,O recurso deve ser encaminhado ao Coordenador ...,6


In [5]:
model = SentenceTransformer('rufimelo/Legal-BERTimbau-sts-base')

In [6]:
encoded_data = model.encode(df.question.tolist())
encoded_data = np.asarray(encoded_data.astype('float32'))
index = faiss.IndexIDMap(faiss.IndexFlatL2(768))
index.add_with_ids(encoded_data, np.array(range(0, len(df))))
faiss.write_index(index, 'database_context.index')